This notebook demonstrates the worklflow of a simple image classification task.
We will go through all the pipeline steps: downloading the model, generating the Intermediate Representation (IR) using the Model Optimizer, running inference in Python, and parsing and interpretating the output results.

To demonstrate the scenario, we will use the pre-trained SquezeNet V1.1 Caffe\* model. SqueezeNet is a pretty accurate and at the same time lightweight network. For more information about the model, please visit <a href="https://github.com/DeepScale/SqueezeNet/">GitHub</a> page and refer to original <a href="https://arxiv.org/abs/1602.07360">SqueezeNet paper</a>.

Follow the steps to perform image classification with the SquezeNet V1.1 model:

**1. Download the model files:** 

In [ ]:
%%bash
echo "Downloading deploy.protxt ..."
if [ -f deploy.prototxt ]; then 
    echo "deploy.protxt file already exists. Downloading skipped"
else
    wget https://raw.githubusercontent.com/DeepScale/SqueezeNet/a47b6f13d30985279789d08053d37013d67d131b/SqueezeNet_v1.1/deploy.prototxt -q
    echo "Finished!"
fi

In [ ]:
%%bash
! echo "Downloading squeezenet_v1.1.caffemodel ..."
if [ -f squeezenet_v1.1.caffemodel ]; then
    echo "squeezenet_v1.1.caffemodel file already exists. Download skipped"
else
    wget https://github.com/DeepScale/SqueezeNet/raw/a47b6f13d30985279789d08053d37013d67d131b/SqueezeNet_v1.1/squeezenet_v1.1.caffemodel -q
    echo "Finished!"
fi

**Run the following command to see the model files:**

In [ ]:
!ls -la

* `deploy.prototxt` contains the network toplogy description in text format. 
* `squeezenet_v1.1.caffemodel` contains weights for all network layers

**2. Optimize and convert the model from intial Caffe representation to the IR representation, which is required for scoring the model using Inference Engine. To convert and optimize the model, use the Model Optimizer command line tool.**

To locate Model Optimizer scripts, specify the path to the Model Optimizer root directory in the `MO_ROOT` variable in the cell bellow and then run it (If you use the installed OpenVINO&trade; package, you can find the Model Optimizer in `<INSTALLATION_ROOT_DIR>/deployment_tools/model_optimizer`).

In [ ]:
%%bash
MO_ROOT=/localdisk/repos/model-optimizer-tensorflow/
echo $MO_ROOT
python3 $MO_ROOT/mo.py --input_model squeezenet_v1.1.caffemodel --input_proto deploy.prototxt

**3. Now, you have the SqueezeNet model converted to the IR, and you can infer it.**

a. First, import required modules:

In [ ]:
from openvino.inference_engine import IENetwork, IEPlugin
import numpy as np
import cv2
import logging as log
from time import time
import sys
import glob
import os
from matplotlib import pyplot as plt
%matplotlib inline

b. Initialize required constants:

In [ ]:
# Configure logging format
log.basicConfig(format="[ %(levelname)s ] %(message)s", level=log.INFO, stream=sys.stdout)

# Path to IR model files
MODEL_XML = "./squeezenet_v1.1.xml"
MODEL_BIN = "./squeezenet_v1.1.bin"

# Target device to run inference
TARGET_DEVICE = "CPU"

# Folder with input images for the model
IMAGES_FOLDER = "./images"

# File containing information about classes names 
LABELS_FILE = "./image_net_synset.txt"

# Number of top prediction results to parse
NTOP = 5

# Required batch size - number of images which will be processed in parallel
BATCH = 4

c. Create a plugin instance for the specified target device  
d. Read the IR files and create an `IENEtwork` instance

In [ ]:
plugin = IEPlugin(TARGET_DEVICE)
net = IENetwork(model=MODEL_XML, weights=MODEL_BIN)

e. Set the network batch size to the constatns specified above. 

Batch size is an "amount" of input data that will be infered in parallel. In this cases it is a number of images, which will be classified in parallel. 

You can set the network batch size using one of the following options:
1. On the IR generation stage, run the Model Optimizer with `-b` command line option. For example, to generate the IR with batch size equal to 4, add `-b 4` to Model Optimizer command line options. By default, it takes the batch size from the original network in framework representation (usually, it is equal to 1, but in this case, the original Caffe model is provided with the batch size equal to 10). 
2. Use Inference Engine after reading IR. We will use this option.

To set the batch size with the Inference Engine:

In [ ]:
log.info("Current network batch size is {}, will be changed to {}".format(net.batch_size, BATCH))
net.batch_size = BATCH

f. After setting batch size, you can get required information about network input layers.
To preprocess input images, you need to know input layer shape.

`inputs` property of `IENetwork` returns the dicitonary with input layer names and `InputInfo` objects, which contain information about an input layer including its shape.

SqueezeNet is a single-input toplogy, so to get the input layer name and its shape, you can get the first item from the `inputs` dictionary:

In [ ]:
input_layer = next(iter(net.inputs))
n,c,h,w = net.inputs[input_layer].shape
layout = net.inputs[input_layer].layout
log.info("Network input layer {} has shape {} and layout {}".format(input_layer, (n,c,h,w), layout))

So what do the shape and layout mean?  
Layout will helps to interprete the shape dimsesnions meaning. 

`NCHW` input layer layout means:
* the fisrt dimension of an input data is a batch of **N** images processed in parallel 
* the second dimension is a numnber of **C**hannels expected in the input images
* the third and the forth are a spatial dimensions - **H**eight and **W**idth of an input image

Our shapes means that the network expects four 3-channel images running in parallel with size 227x227.

g. Read and preprocess input images.

For it, go to `IMAGES_FOLDER`, find all `.bmp` files, and take four images for inference:

In [ ]:
search_pattern = os.path.join(IMAGES_FOLDER, "*.bmp")
images = glob.glob(search_pattern)[:BATCH]
log.info("Input images:\n {}".format("\n".join(images)))

Now you can read and preprocess the image files and create an array with input blob data.

For preprocessing, you must do the following:
1. Resize the images to fit the HxW input dimenstions.
2. Transpose the HWC layout.

Transposing is tricky and not really obvious.
As you alredy saw above, the network has the `NCHW` layout, so each input image should be in `CHW` format. But by deafult, OpenCV\* reads images in the `HWC` format. That is why you have to swap the axes using the `numpy.transpose()` function:

In [ ]:
input_data = np.ndarray(shape=(n, c, h, w))
orig_images = [] # Will be used to show image in notebook
for i, img in enumerate(images):
    image = cv2.imread(img)
    orig_images.append(image)
    if image.shape[:-1] != (h, w):
        log.warning("Image {} is resized from {} to {}".format(img, image.shape[:-1], (h, w)))
        image = cv2.resize(image, (w, h))
    image = image.transpose((2, 0, 1))  # Change data layout from HWC to CHW
    input_data[i] = image

i. Infer the model model to classify input images:

1. Load the `IENetwork` object to the plugin to create `ExectuableNEtwork` object.    
2. Start inference using the `infer()` function specifying dictionary with input layer name and prepared data as an argument for the function.     
3. Measure inference time in miliseconds and calculate throughput metric in frames-per-second (FPS).

In [ ]:
exec_net = plugin.load(net)
t0 = time()
res_map = exec_net.infer({input_layer: input_data})
inf_time = (time() - t0) * 1000 
fps = BATCH * inf_time 
log.info("Inference time: {} ms.".format(inf_time))
log.info("Throughput: {} fps.".format(fps))

**4. After the inference, you need to parse and interpretate the inference results.**

First, you need to see the shape of the network output layer. It can be done in similar way as for the inputs, but here you need to call `outputs` property of `IENetwork` object:

In [ ]:
output_layer = next(iter(net.outputs))
n,c,h,w = net.outputs[output_layer].shape
layout = net.outputs[output_layer].layout
log.info("Network output layer {} has shape {} and layout {}".format(output_layer, (n,c,h,w), layout))

It is not a common case for classification netowrks to have output layer with *NCHW* layout. Usually, it is just *NC*. However, in this case, the last two dimensions are just a feature of the network and do not have much sense. Ignore them as you will remove  them on the final parsing stage. 

What are the first and second dimensions of the output layer?    
* The first dimension is a batch. We precoessed four images, and the prediction result for a particular image is stored in the first dimension of the output array. For example, prediction results for the third image is `res[2]` (since numeration starts from 0).
* The second dimension is an array with normalized probabilities (from 0 to 1) for each class. This network is trained using the <a href="http://image-net.org/index">ImageNet</a> dataset with 1000 classes. Each `n`-th value in the output data for a certain image represent the probability of the image belonging to the `n`-th class. 

To parse the output results:

a. Read the `LABELS_FILE`, which maps the class ID to human-readable class names:

In [ ]:
with open(LABELS_FILE, 'r') as f:
    labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]


b. Parse the output array with prediction results. The parsing algorith is the following:
0. Squeeze the last two "extra" dimensions of the output data.
1. Iterate over all batches.
2. Sort the probabilities vector descendingly to get `NTOP` classes with the highest probabilities (by default, the `numpy.argsort` sorts the data in the ascending order, but using the array slicing `[::-1]`, you can reverse the data order).
3. Map the `NTOP` probabilities to the corresponding labeles in `labeles_map`.

For the vizualization, you also need to store top-1 class and probability.

In [ ]:
top1_res = [] # will be used for the visualization
res = np.squeeze(res_map[output_layer])
log.info("Top {} results: ".format(NTOP))
for i, probs in enumerate(res):
    top_ind = np.argsort(probs)[-NTOP:][::-1]
    print("Image {}".format(images[i]))
    top1_ind = top_ind[0]
    top1_res.append((labels_map[top1_ind], probs[top1_ind]))
    for id in top_ind:
        print("label: {}   probability: {:.2f}% ".format(labels_map[id], probs[id] * 100))
    print("\n")

The code above prints the results as plain text.   
You can also use OpenCV\* to visualize the results using the `orig_images` and `top1_res` variables, which you created during images reading and results parsing:

In [ ]:
plt.clf()
for i, img in enumerate(orig_images):
    label_str = "{}".format(top1_res[i][0].split(',')[0])
    prob_str = "{:.2f}%".format(top1_res[i][1])
    cv2.putText(img, label_str, (5, 15), cv2.FONT_HERSHEY_COMPLEX, 0.6, (220,100,10), 1)
    cv2.putText(img, prob_str, (5, 35), cv2.FONT_HERSHEY_COMPLEX, 0.6, (220,100,10), 1)
    plt.figure()
    plt.axis("off")
    
    # We have to convert colors, because matplotlib expects an image in RGB color format 
    # but by default, the OpenCV read images in BRG format
    im_to_show = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(im_to_show)